# API (121 Applied Mathematics and Computer Science)

original by Grandmaster Дуркин Анатолий Альбертович

## Что за API

API (application programming interface, интерфейс программирования приложения) — программный интерфейс, то есть описание способов взаимодействия одной компьютерной программы с другими. Это своеобразный аналог пользовательского интерфеса, но для программ.

API предоставляет набор функций, которые упрощают взаимодействие с системой. API определяется и создаётся разработчиком, в связи с чем у всех есть свои особенности, которые надо изучать, к которым надо привыкать. К сожалению, не всегда документации позволяют полностью понять возможности того или иного API.

Чаще всего API это использование функций библиотек, которые возвращают разные объекты. Так, иногда даже создают сторонние библиотеки для работы с API - для Telegram помимо своего собственного API можно найти библиотеки pyrogram и telethon, которые упрощают взаимодействие. Но иногда бывает иначе. Например, в том варианте, который мы сейчас рассмотрим.

In [3]:
%pip install -r requirements.txt

  Using cached pandas-2.3.3-cp314-cp314-manylinux_2_24_x86_64.manylinux_2_28_x86_64.whl.metadata (91 kB)
  Using cached numpy-2.4.0-cp314-cp314-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (6.6 kB)
  Using cached jupyter-1.1.1-py2.py3-none-any.whl.metadata (2.0 kB)
ERROR: Ignored the following versions that require a different python version: 1.21.2 Requires-Python >=3.7,<3.11; 1.21.3 Requires-Python >=3.7,<3.11; 1.21.4 Requires-Python >=3.7,<3.11; 1.21.5 Requires-Python >=3.7,<3.11; 1.21.6 Requires-Python >=3.7,<3.11; 1.26.0 Requires-Python >=3.9,<3.13; 1.26.1 Requires-Python >=3.9,<3.13
ERROR: Could not find a version that satisfies the requirement json (from versions: none)
ERROR: No matching distribution found for json
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import requests
import json

ModuleNotFoundError: No module named 'pandas'

## API hh.ru

API hh.ru позволяет создавать приложения, интегрирующиеся с сайтом. Но мы используем это для того, чтобы достать информацию с сайта и использовать её для анализа.

Документация: https://dev.hh.ru/

В документации очень подробно рассказано, как пользоваться API, к тому же, она на русском языке. Там пишут, что для работы приложения нужно авторизоваться и описывают весь процесс. Вы можете изучить это, но сейчас мы будем доставать данные без авторизации.

Допустим, мы хотим получить данные по всем вакансиям, как если бы искали на сайте вакансии по должности "Data Scientist".

In [ ]:
url = 'https://api.hh.ru/vacancies'
params = {'text': 'Data Scientist', 'per_page': '20', 'page': '1'}
src = requests.get(url, params=params)
data = src.json()

Мы указали адрес, откуда получать данные. Здесь взаимодействие происходит не через библиотеку или функции, а при обращении к определенному адресу.

Также мы указываем параметры: должность, которую ищем, сколько вакансий на странице отображать и какую страницу читаем.

А дальше мы просто получаем данные по указанным параметрам и превращаем их в удобный нам формат. В данном случае данные возвращаются нам в формате JSON - очень популярном для общения сервисов в сети - поэтому в таком виде их и обрабатываем.

Посмотрим, что мы получили.

In [ ]:
data

Мы получили вакансии, как и запрашивали. Вот только теперь они лежат в словарях, которые нам предстоит разобрать. Описанипе всех тегов вы можете найти в документации.

Итак, посмотрим, какие ключи есть в нашем словаре.

In [ ]:
data.keys()

Здесь указаны параметры, которые мы передавали (относительно страниц и числа вакансий на них), а также прочие основные параметры. Но нас интересует только список с вакансиями, он лежит с ключом `items`.

In [ ]:
len(data['items'])

Тут мы оценили размер списка, лежащего под указанным ключом. В нём действительно 20 элементов, как мы и запрашивали. Посмотрим на первый.

In [ ]:
data['items'][0]

## Параметры

Мы видим большое количество параметров, свойств, описание которых (подробное и на русском) есть в документации. Отсюда многое можно взять в нашу таблицу для анализа. Я возьму лишь пару параметров для демонстрации.

In [ ]:
vacancy = data['items'][0]

In [ ]:
vacancy['name']

In [ ]:
vacancy['id']

In [ ]:
vacancy['salary']

Вот тут могут начинаться первые проблемы. Данных о зарплате может и не быть вовсе, а могут быть неполные. Тут стоит обрабатывать данные аккуратно.

Подберём две вакансии, одну с зарплатой, другую без, и посмотрим разницу.

In [ ]:
# с зарплатой
vacancy_s = [v for v in data['items'] if v['salary']][0]
# без зарплаты
vacancy_w = [v for v in data['items'] if not v['salary']][0]

Дадите объяснение моим действиям? Если не очень понятно, то пустое значение `None` приравнивается к `False`, поэтому условие выглядит именно так. А как бы сделали вы?

In [ ]:
vacancy_w['salary']

Тут и правда не указаны зарплаты. Что ж, посмотрим на вторую вакансию.

In [ ]:
vacancy_s['salary']

Внутри не просто число, внутри лежит целый словарь (если что-то пошло не так, выгрузите список вакансий снова или увеличьте размер выгрузки, возможно, в этом нет вакансий с зарплатой). А в нём указаны границы зарплатной вилки, валюта и даже указание, до или после вычета налогов.

Как бы вы поступили с такими данными? Я возьму только одно из значений - нижнюю границу зарплаты. Но это для демонстрации, ваше решение явно будет отличаться.

In [ ]:
vacancy_s['salary']['from']

Это всё хорошо, но как же достать это значение, если этого словаря не будет вообще? Конечно, проверить на наличие. Давайте напишем небольшой цикл, пробежав по имеющимся у нас вакансиям.

In [ ]:
for vacancy in data['items']:
    if vacancy['salary']:
        print(vacancy['salary']['from'])
    else:
        print('Не указано')

Сможем ли мы сократить это условие? Конечно, воспользуемся тернарным оператором:

In [ ]:
for vacancy in data['items']:
    print(vacancy['salary']['from'] if vacancy['salary'] else 'Не указано')

Прекрасно! А главное, кратко и понятно.

## Собираем таблицу

Попробуем создать небольшую таблицу для данных. Я создам совсем небольшую, чтобы посмотреть, как это работает.

In [ ]:
df = pd.DataFrame(columns=['name', 'salary_from'])

for vacancy in data['items']:
    id = vacancy['id']
    name = vacancy['name']
    salary_from = vacancy['salary']['from'] if vacancy['salary'] else None
    df.loc[id] = [name, salary_from]

In [ ]:
df

Да, табличка и правда получилась!

Конечно, я оставил для вас самое интересное. Как минимум, примените бритву Оккама - тут слишком много лишних сущностей, можно и короче. А как максимум...

## Парсим интересные вакансии

Как максимум я хочу, чтобы вы собрали большую таблицу по вакансиям и проанализировали её. При этом выберите ту должность, которая интересно лично вам.

Предлагаю такие шаги:

1. Какие данные из вакансии заслуживают внимания? Что стоит доставать и помещать в таблицу? Уделите особое внимание численным параметрам - с остальными мы пока не особо учились работать.
2. Какие данные могут быть проблемными? Где-то могут встречаться пропуски, как в зарплатах, их нужно обработать.
3. Как собрать больше вакансий? Больше 100 на странице не получится, а страниц может быть несколько. Надо собрать данные со всех.
4. Какой базовый анализ можно провести? Интересно, какие зарплаты предлагают по должностям, в каких городах сколько вакансий, и прочее, прочее, прочее...

Уверен, это не составит большого труда, у вас всё получится!

In [ ]:
# Ваш код